# Preparación de Datos para Modelado

Este notebook realiza la preparación final de los datos para el entrenamiento de modelos de machine learning:

- Cargar librerías necesarias
- Identificar y codificar variables categóricas
- Dividir el conjunto de datos en entrenamiento y prueba
- Guardar los conjuntos de datos procesados

**Nota:** Se asume que los datos ya han sido limpiados previamente.

## 1. Cargar Librerías Requeridas

Importar todas las librerías necesarias para la preparación de datos y codificación de variables.

In [1]:
import pandas as pd
import numpy as np
import yaml
import os
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler
import joblib
from datetime import datetime

# Suprimir warnings para una salida más limpia
warnings.filterwarnings('ignore')

# Configuración de pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("✓ Librerías importadas exitosamente!")
print(f"Pandas versión: {pd.__version__}")
print(f"NumPy versión: {np.__version__}")

✓ Librerías importadas exitosamente!
Pandas versión: 2.3.2
NumPy versión: 2.3.2


## 2. Cargar Configuración y Datos

Cargar los parámetros desde el archivo `params.yaml` y el conjunto de datos limpio.

In [2]:
# Cargar configuración desde params.yaml
config = yaml.safe_load(open("../params.yaml"))["prepare"]

print("Configuración cargada:")
print("=" * 40)
for key, value in config.items():
    print(f"{key}: {value}")

# Extraer parámetros
input_path = "../" + config["input_path"]
output_path_train = "../" + config["output_path_train"]
output_path_test = "../" + config["output_path_test"]
test_split = config["split"]
random_seed = config["seed"]

print(f"\n✓ Configuración cargada exitosamente!")
print(f"✓ División de test: {test_split*100}%")
print(f"✓ Semilla aleatoria: {random_seed}")

Configuración cargada:
input_path: data/prep/dataset_cleaned.csv
output_path_train: data/train
output_path_test: data/test
split: 0.2
seed: 20250901

✓ Configuración cargada exitosamente!
✓ División de test: 20.0%
✓ Semilla aleatoria: 20250901


In [3]:
# Cargar el conjunto de datos limpio
print(f"Cargando datos desde: {input_path}")

if os.path.exists(input_path):
    df = pd.read_csv(input_path)
    print(f"✓ Datos cargados exitosamente!")
    print(f"✓ Forma del conjunto de datos: {df.shape}")
    print(f"✓ Total de registros: {len(df):,}")
    print(f"✓ Total de características: {df.shape[1]}")
else:
    raise FileNotFoundError(f"Archivo no encontrado: {input_path}")

# Establecer semilla para reproducibilidad
np.random.seed(random_seed)
print(f"\n✓ Semilla aleatoria establecida: {random_seed}")

Cargando datos desde: ../data/prep/dataset_cleaned.csv
✓ Datos cargados exitosamente!
✓ Forma del conjunto de datos: (113999, 16)
✓ Total de registros: 113,999
✓ Total de características: 16

✓ Semilla aleatoria establecida: 20250901


## 3. Exploración de Variables

Analizar el conjunto de datos para identificar tipos de variables y su distribución.

In [4]:
# Mostrar información general del conjunto de datos
print("INFORMACIÓN GENERAL DEL CONJUNTO DE DATOS")
print("=" * 50)
print(f"Forma: {df.shape}")
print(f"\nPrimeras 5 filas:")
display(df.head())

print(f"\nInformación de tipos de datos:")
print(df.info())

INFORMACIÓN GENERAL DEL CONJUNTO DE DATOS
Forma: (113999, 16)

Primeras 5 filas:


,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic



Información de tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113999 entries, 0 to 113998
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   popularity        113999 non-null  int64  
 1   duration_ms       113999 non-null  int64  
 2   explicit          113999 non-null  bool   
 3   danceability      113999 non-null  float64
 4   energy            113999 non-null  float64
 5   key               113999 non-null  int64  
 6   loudness          113999 non-null  float64
 7   mode              113999 non-null  int64  
 8   speechiness       113999 non-null  float64
 9   acousticness      113999 non-null  float64
 10  instrumentalness  113999 non-null  float64
 11  liveness          113999 non-null  float64
 12  valence           113999 non-null  float64
 13  tempo             113999 non-null  float64
 14  time_signature    113999 non-null  int64  
 15  track_genre       113999 non-null  o

In [5]:
df.describe(include='all')

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
count,113999.000000,1.139990e+05,113999,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999.000000,113999
unique,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114
top,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,acoustic
freq,NaN,NaN,104252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000
mean,33.238827,2.280312e+05,NaN,0.566801,0.641383,5.309126,-8.258950,0.637558,0.084652,0.314907,0.156051,0.213554,0.474066,122.147695,3.904034,NaN
std,22.304959,1.072961e+05,NaN,0.173543,0.251530,3.559999,5.029357,0.480708,0.105733,0.332522,0.309556,0.190378,0.259261,29.978290,0.432623,NaN
min,0.000000,8.586000e+03,NaN,0.000000,0.000000,0.000000,-49.531000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,17.000000,1.740660e+05,NaN,0.456000,0.472000,2.000000,-10.013000,0.000000,0.035900,0.016900,0.000000,0.098000,0.260000,99.218500,4.000000,NaN
50%,35.000000,2.129060e+05,NaN,0.580000,0.685000,5.000000,-7.004000,1.000000,0.048900,0.169000,0.000042,0.132000,0.464000,122.017000,4.000000,NaN
75%,50.000000,2.615060e+05,NaN,0.695000,0.854000,8.000000,-5.003000,1.000000,0.084500,0.597500,0.049000,0.273000,0.683000,140.071000,4.000000,NaN


In [6]:
# Identificar tipos de variables
print("ANÁLISIS DE TIPOS DE VARIABLES")
print("=" * 50)

# Variables categóricas (object y category)
categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()
categorical_columns += ['key', 'mode', 'time_signature']
print(f"\nVariables categóricas ({len(categorical_columns)}):")
for col in categorical_columns:
    unique_values = df[col].nunique()
    print(f"  - {col}: {df[col].dtype} (valores únicos: {unique_values})")

# Variables numéricas
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
# Excluir columnas numéricas que están en categorical_columns
numeric_columns = [col for col in numeric_columns if col not in categorical_columns]
print(f"Variables numéricas ({len(numeric_columns)}):")
for col in numeric_columns:
    print(f"  - {col}: {df[col].dtype}")

# Variables booleanas
boolean_columns = df.select_dtypes(include=['bool']).columns.tolist()
print(f"\nVariables booleanas ({len(boolean_columns)}):")
for col in boolean_columns:
    print(f"  - {col}: {df[col].dtype}")

ANÁLISIS DE TIPOS DE VARIABLES

Variables categóricas (4):
  - track_genre: object (valores únicos: 114)
  - key: int64 (valores únicos: 12)
  - mode: int64 (valores únicos: 2)
  - time_signature: int64 (valores únicos: 5)
Variables numéricas (11):
  - popularity: int64
  - duration_ms: int64
  - danceability: float64
  - energy: float64
  - loudness: float64
  - speechiness: float64
  - acousticness: float64
  - instrumentalness: float64
  - liveness: float64
  - valence: float64
  - tempo: float64

Variables booleanas (1):
  - explicit: bool


## 4. Normalización de Variables Numéricas

Analizar y normalizar las variables numéricas para mejorar el rendimiento de los algoritmos de machine learning que son sensibles a la escala de las características.

In [7]:
# Análisis estadístico de variables numéricas
columns_to_normalize = ['duration_ms','loudness', 'tempo']

print("ANÁLISIS ESTADÍSTICO DE VARIABLES NUMÉRICAS A NORMALIZAR")
print("=" * 50)

# Estadísticas descriptivas de variables numéricas
numeric_stats = df[columns_to_normalize].describe()
display(numeric_stats)

print(f"\nRangos de valores para cada variable numérica a normalizar:")
print("-" * 50)
for col in columns_to_normalize:
    min_val = df[col].min()
    max_val = df[col].max()
    mean_val = df[col].mean()
    std_val = df[col].std()
    print(f"{col}:")
    print(f"  • Rango: [{min_val:.3f}, {max_val:.3f}]")
    print(f"  • Media: {mean_val:.3f}, Desv. Estándar: {std_val:.3f}")
    ratio = max_val / min_val if min_val != 0 else 'Inf'
    if isinstance(ratio, str):
        print(f"  • Ratio (max/min): {ratio}")
    else:
        print(f"  • Ratio (max/min): {ratio:.2f}")
    print()

ANÁLISIS ESTADÍSTICO DE VARIABLES NUMÉRICAS A NORMALIZAR


,duration_ms,loudness,tempo
count,1.139990e+05,113999.000000,113999.000000
mean,2.280312e+05,-8.258950,122.147695
std,1.072961e+05,5.029357,29.978290
min,8.586000e+03,-49.531000,0.000000
25%,1.740660e+05,-10.013000,99.218500
50%,2.129060e+05,-7.004000,122.017000
75%,2.615060e+05,-5.003000,140.071000
max,5.237295e+06,4.532000,243.372000



Rangos de valores para cada variable numérica a normalizar:
--------------------------------------------------
duration_ms:
  • Rango: [8586.000, 5237295.000]
  • Media: 228031.153, Desv. Estándar: 107296.058
  • Ratio (max/min): 609.98

loudness:
  • Rango: [-49.531, 4.532]
  • Media: -8.259, Desv. Estándar: 5.029
  • Ratio (max/min): -0.09

tempo:
  • Rango: [0.000, 243.372]
  • Media: 122.148, Desv. Estándar: 29.978
  • Ratio (max/min): Inf



In [8]:
# Determinación de estrategia de normalización
if columns_to_normalize:
    print("DETERMINACIÓN DE ESTRATEGIA DE NORMALIZACIÓN")
    print("=" * 50)
    
    # Análisis de distribuciones para determinar el mejor método
    variables_to_normalize = []
    normalization_strategy = {}
    
    for col in columns_to_normalize:
        # Calcular estadísticas para determinar el método apropiado
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        
        # Detectar outliers usando IQR
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]
        outlier_percentage = len(outliers) / len(df) * 100
        
        # Determinar el rango de valores
        min_val = df[col].min()
        max_val = df[col].max()
        range_ratio = max_val / min_val if min_val > 0 else float('inf')
        
        # Decidir método de normalización
        if outlier_percentage > 5:  # Si hay muchos outliers, usar RobustScaler
            method = "RobustScaler"
            scaler = RobustScaler()
        elif range_ratio > 100:  # Si el rango es muy amplio, usar StandardScaler
            method = "StandardScaler"
            scaler = StandardScaler()
        else:  # Para rangos moderados, usar MinMaxScaler
            method = "MinMaxScaler"
            scaler = MinMaxScaler()
        
        variables_to_normalize.append(col)
        normalization_strategy[col] = {'method': method, 'scaler': scaler, 'outlier_pct': outlier_percentage}
        
        print(f"✓ {col}: {method}")
        print(f"  • Outliers: {outlier_percentage:.1f}%")
        print(f"  • Ratio rango: {range_ratio:.2f}")
        print()
    
    print(f"Total de variables a normalizar: {len(variables_to_normalize)}")
else:
    variables_to_normalize = []
    normalization_strategy = {}
    print("✓ No hay variables numéricas para normalizar.")

DETERMINACIÓN DE ESTRATEGIA DE NORMALIZACIÓN
✓ duration_ms: StandardScaler
  • Outliers: 4.9%
  • Ratio rango: 609.98

✓ loudness: RobustScaler
  • Outliers: 5.4%
  • Ratio rango: inf

✓ tempo: StandardScaler
  • Outliers: 0.5%
  • Ratio rango: inf

Total de variables a normalizar: 3


In [9]:
# Aplicar normalización a las variables numéricas
df_normalized = df.copy()
scalers = {}  # Diccionario para guardar los scalers

if variables_to_normalize:
    print("APLICANDO NORMALIZACIÓN")
    print("=" * 50)
    
    for col in variables_to_normalize:
        method = normalization_strategy[col]['method']
        scaler = normalization_strategy[col]['scaler']
        
        # Aplicar normalización
        original_values = df[col].values.reshape(-1, 1)
        normalized_values = scaler.fit_transform(original_values)
        df_normalized[col] = normalized_values.flatten()
        
        # Guardar el scaler para uso futuro
        scalers[f'{col}_scaler'] = scaler
        
        # Mostrar estadísticas antes y después
        original_mean = df[col].mean()
        original_std = df[col].std()
        normalized_mean = df_normalized[col].mean()
        normalized_std = df_normalized[col].std()
        
        print(f"✓ {col} ({method}):")
        print(f"  • Antes: μ={original_mean:.3f}, σ={original_std:.3f}")
        print(f"  • Después: μ={normalized_mean:.3f}, σ={normalized_std:.3f}")
        print(f"  • Rango después: [{df_normalized[col].min():.3f}, {df_normalized[col].max():.3f}]")
        print()
    
    print(f"✓ Normalización completada!")
    print(f"✓ Variables normalizadas: {len(variables_to_normalize)}")
    print(f"✓ Scalers guardados: {len(scalers)}")
else:
    print("✓ No hay variables para normalizar.")

APLICANDO NORMALIZACIÓN
✓ duration_ms (StandardScaler):
  • Antes: μ=228031.153, σ=107296.058
  • Después: μ=-0.000, σ=1.000
  • Rango después: [-2.045, 46.687]

✓ loudness (RobustScaler):
  • Antes: μ=-8.259, σ=5.029
  • Después: μ=-0.250, σ=1.004
  • Rango después: [-8.488, 2.303]

✓ tempo (StandardScaler):
  • Antes: μ=122.148, σ=29.978
  • Después: μ=0.000, σ=1.000
  • Rango después: [-4.075, 4.044]

✓ Normalización completada!
✓ Variables normalizadas: 3
✓ Scalers guardados: 3


## 5. Identificar y Codificar Variables Categóricas

Procesar las variables categóricas utilizando técnicas de codificación apropiadas.

In [10]:
# Análisis detallado de variables categóricas
if categorical_columns:
    print("ANÁLISIS DETALLADO DE VARIABLES CATEGÓRICAS")
    print("=" * 50)
    
    for col in categorical_columns:
        unique_count = df[col].nunique()
        sample_values = df[col].value_counts().head(10)
        
        print(f"\nVariable: {col}")
        print(f"Valores únicos: {unique_count}")
        print(f"Top 10 valores más frecuentes:")
        print(sample_values)
        print("-" * 30)
else:
    print("✓ No se encontraron variables categóricas para procesar.")

ANÁLISIS DETALLADO DE VARIABLES CATEGÓRICAS

Variable: track_genre
Valores únicos: 114
Top 10 valores más frecuentes:
track_genre
acoustic             1000
afrobeat             1000
psych-rock           1000
progressive-house    1000
power-pop            1000
pop                  1000
pop-film             1000
piano                1000
party                1000
pagode               1000
Name: count, dtype: int64
------------------------------

Variable: key
Valores únicos: 12
Top 10 valores más frecuentes:
key
7     13244
0     13061
2     11644
9     11313
1     10772
5      9368
11     9282
4      9008
6      7921
10     7456
Name: count, dtype: int64
------------------------------

Variable: mode
Valores únicos: 2
Top 10 valores más frecuentes:
mode
1    72681
0    41318
Name: count, dtype: int64
------------------------------

Variable: time_signature
Valores únicos: 5
Top 10 valores más frecuentes:
time_signature
4    101842
3      9195
5      1826
1       973
0       163
Name: co

In [11]:
# Determinar estrategia de codificación para cada variable categórica
if categorical_columns:
    print("ESTRATEGIA DE CODIFICACIÓN")
    print("=" * 50)
    
    # Separar variables por cardinalidad
    low_cardinality = []  # Para One-Hot Encoding (<=10 categorías)
    high_cardinality = [] # Para Label Encoding (>10 categorías)
    
    for col in categorical_columns:
        unique_count = df[col].nunique()
        if unique_count <= 10:
            low_cardinality.append(col)
            print(f"✓ {col}: One-Hot Encoding (cardinalidad: {unique_count})")
        else:
            high_cardinality.append(col)
            print(f"✓ {col}: Label Encoding (cardinalidad: {unique_count})")
    
    print(f"\nResumen:")
    print(f"Variables para One-Hot Encoding: {len(low_cardinality)}")
    print(f"Variables para Label Encoding: {len(high_cardinality)}")
else:
    low_cardinality = []
    high_cardinality = []
    print("✓ No hay variables categóricas para codificar.")

ESTRATEGIA DE CODIFICACIÓN
✓ track_genre: Label Encoding (cardinalidad: 114)
✓ key: Label Encoding (cardinalidad: 12)
✓ mode: One-Hot Encoding (cardinalidad: 2)
✓ time_signature: One-Hot Encoding (cardinalidad: 5)

Resumen:
Variables para One-Hot Encoding: 2
Variables para Label Encoding: 2


In [12]:
# Aplicar codificación a variables categóricas
df_encoded = df_normalized.copy()  # Usar el dataframe normalizado
encoders = {}  # Diccionario para guardar los encoders

print("APLICANDO CODIFICACIÓN")
print("=" * 50)

# Label Encoding para variables de alta cardinalidad
if high_cardinality:
    print("Aplicando Label Encoding...")
    for col in high_cardinality:
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df_normalized[col].astype(str))
        encoders[f'{col}_label_encoder'] = le
        print(f"  ✓ {col}: {df_normalized[col].nunique()} categorías → valores numéricos")

# One-Hot Encoding para variables de baja cardinalidad
if low_cardinality:
    print("\nAplicando One-Hot Encoding...")
    for col in low_cardinality:
        # Crear variables dummy
        dummies = pd.get_dummies(df_normalized[col], prefix=col, drop_first=True)
        
        # Remover columna original y agregar las nuevas
        df_encoded = df_encoded.drop(columns=[col])
        df_encoded = pd.concat([df_encoded, dummies], axis=1)
        
        # Guardar las columnas creadas para referencia
        encoders[f'{col}_dummy_columns'] = dummies.columns.tolist()
        print(f"  ✓ {col}: {df_normalized[col].nunique()} categorías → {len(dummies.columns)} variables dummy")

print(f"\n✓ Codificación completada!")
print(f"✓ Forma después de normalización: {df_normalized.shape}")
print(f"✓ Forma después de codificación: {df_encoded.shape}")
print(f"✓ Nuevas características creadas: {df_encoded.shape[1] - df_normalized.shape[1]}")

APLICANDO CODIFICACIÓN
Aplicando Label Encoding...
  ✓ track_genre: 114 categorías → valores numéricos
  ✓ key: 12 categorías → valores numéricos

Aplicando One-Hot Encoding...
  ✓ mode: 2 categorías → 1 variables dummy
  ✓ time_signature: 5 categorías → 4 variables dummy

✓ Codificación completada!
✓ Forma después de normalización: (113999, 16)
✓ Forma después de codificación: (113999, 19)
✓ Nuevas características creadas: 3


## 6. División en Conjuntos de Entrenamiento y Prueba

Dividir el conjunto de datos codificado en conjuntos de entrenamiento y prueba según los parámetros configurados.

In [13]:
# Verificar si existe una columna objetivo (target)
print("IDENTIFICACIÓN DE VARIABLE OBJETIVO")
print("=" * 50)

target_column = 'popularity'
print(f"✓ Variable objetivo seleccionada: {target_column}")

# Separar características (X) y variable objetivo (y)
X = df_encoded.drop(columns=[target_column])
y = df_encoded[target_column]

print(f"\nForma de características (X): {X.shape}")
print(f"Forma de variable objetivo (y): {y.shape}")
print(f"\nDistribución de la variable objetivo:")
print(y.value_counts().sort_index())

IDENTIFICACIÓN DE VARIABLE OBJETIVO
✓ Variable objetivo seleccionada: popularity

Forma de características (X): (113999, 18)
Forma de variable objetivo (y): (113999,)

Distribución de la variable objetivo:
popularity
0      16019
1       2140
2       1036
3        585
4        389
       ...  
96         7
97         8
98         7
99         1
100        2
Name: count, Length: 101, dtype: int64


In [14]:
# Realizar la división train/test
print("DIVISIÓN EN CONJUNTOS DE ENTRENAMIENTO Y PRUEBA")
print("=" * 50)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=test_split, 
    random_state=random_seed,
    #stratify=y if len(y.unique()) > 1 and len(y.unique()) < len(y) else None
)

print(f"✓ División completada con semilla: {random_seed}")
print(f"✓ Tamaño de división de prueba: {test_split*100}%")
print(f"\nTamaños de conjuntos:")
print(f"  - Entrenamiento: {len(X_train):,} registros ({len(X_train)/len(X)*100:.1f}%)")
print(f"  - Prueba: {len(X_test):,} registros ({len(X_test)/len(X)*100:.1f}%)")

print(f"\nDistribución en conjunto de entrenamiento:")
print(y_train.value_counts().sort_index())

print(f"\nDistribución en conjunto de prueba:")
print(y_test.value_counts().sort_index())

DIVISIÓN EN CONJUNTOS DE ENTRENAMIENTO Y PRUEBA
✓ División completada con semilla: 20250901
✓ Tamaño de división de prueba: 20.0%

Tamaños de conjuntos:
  - Entrenamiento: 91,199 registros (80.0%)
  - Prueba: 22,800 registros (20.0%)

Distribución en conjunto de entrenamiento:
popularity
0      12783
1       1713
2        844
3        458
4        323
       ...  
96         6
97         7
98         6
99         1
100        2
Name: count, Length: 101, dtype: int64

Distribución en conjunto de prueba:
popularity
0     3236
1      427
2      192
3      127
4       66
      ... 
92       2
95       2
96       1
97       1
98       1
Name: count, Length: 97, dtype: int64


## 7. Guardar Conjuntos de Datos Procesados

Exportar los conjuntos de entrenamiento y prueba a los directorios especificados en la configuración.

In [15]:
# Crear directorios de salida
print("GUARDANDO CONJUNTOS DE DATOS PROCESADOS")
print("=" * 50)

os.makedirs(output_path_train, exist_ok=True)
os.makedirs(output_path_test, exist_ok=True)

print(f"✓ Directorios creados:")
print(f"  - Entrenamiento: {output_path_train}")
print(f"  - Prueba: {output_path_test}")

GUARDANDO CONJUNTOS DE DATOS PROCESADOS
✓ Directorios creados:
  - Entrenamiento: ../data/train
  - Prueba: ../data/test


In [16]:
# Guardar conjunto de entrenamiento
train_features_path = os.path.join(output_path_train, 'X_train.csv')
train_target_path = os.path.join(output_path_train, 'y_train.csv')

X_train.to_csv(train_features_path, index=False)
y_train.to_csv(train_target_path, index=False)

print(f"✓ Conjunto de entrenamiento guardado:")
print(f"  - Características: {train_features_path}")
print(f"  - Variable objetivo: {train_target_path}")
print(f"  - Registros: {len(X_train):,}")
print(f"  - Características: {X_train.shape[1]}")

✓ Conjunto de entrenamiento guardado:
  - Características: ../data/train/X_train.csv
  - Variable objetivo: ../data/train/y_train.csv
  - Registros: 91,199
  - Características: 18


In [17]:
# Guardar conjunto de prueba
test_features_path = os.path.join(output_path_test, 'X_test.csv')
test_target_path = os.path.join(output_path_test, 'y_test.csv')

X_test.to_csv(test_features_path, index=False)
y_test.to_csv(test_target_path, index=False)

print(f"✓ Conjunto de prueba guardado:")
print(f"  - Características: {test_features_path}")
print(f"  - Variable objetivo: {test_target_path}")
print(f"  - Registros: {len(X_test):,}")
print(f"  - Características: {X_test.shape[1]}")

✓ Conjunto de prueba guardado:
  - Características: ../data/test/X_test.csv
  - Variable objetivo: ../data/test/y_test.csv
  - Registros: 22,800
  - Características: 18


In [18]:
# Guardar encoders, scalers y metadata
encoders_path = os.path.join(output_path_train, 'encoders.joblib')
scalers_path = os.path.join(output_path_train, 'scalers.joblib')
metadata_path = os.path.join(output_path_train, 'metadata.yaml')

# Guardar encoders
if encoders:
    joblib.dump(encoders, encoders_path)
    print(f"✓ Encoders guardados: {encoders_path}")

# Guardar scalers
if scalers:
    joblib.dump(scalers, scalers_path)
    print(f"✓ Scalers guardados: {scalers_path}")

# Crear metadata
metadata = {
    'original_shape': df.shape,
    'encoded_shape': df_encoded.shape,
    'target_column': target_column,
    'numeric_columns': numeric_columns,
    'categorical_columns': categorical_columns,
    'low_cardinality_encoded': low_cardinality,
    'high_cardinality_encoded': high_cardinality,
    'train_size': len(X_train),
    'test_size': len(X_test),
    'test_split_ratio': test_split,
    'random_seed': random_seed,
    'feature_count': X_train.shape[1],
    'encoding_date': datetime.now().strftime('%Y-%m-%d')
}

with open(metadata_path, 'w') as f:
    yaml.dump(metadata, f, default_flow_style=False)

print(f"✓ Metadata guardada: {metadata_path}")

✓ Encoders guardados: ../data/train/encoders.joblib
✓ Scalers guardados: ../data/train/scalers.joblib
✓ Metadata guardada: ../data/train/metadata.yaml


## 8. Resumen Final

Resumen completo del proceso de preparación de datos.

In [19]:
# Resumen final del proceso
print("🎉 PREPARACIÓN DE DATOS COMPLETADA EXITOSAMENTE")
print("=" * 60)

print(f"📊 ESTADÍSTICAS GENERALES:")
print(f"  • Conjunto de datos original: {df.shape}")
print(f"  • Conjunto de datos normalizado: {df_normalized.shape}")
print(f"  • Conjunto de datos codificado: {df_encoded.shape}")
print(f"  • Nuevas características creadas: {df_encoded.shape[1] - df.shape[1]}")

print(f"\n🔧 PROCESAMIENTO REALIZADO:")
print(f"  • Variables numéricas: {len(numeric_columns)}")
print(f"  • Variables normalizadas: {len(variables_to_normalize)}")
print(f"  • Variables categóricas procesadas: {len(categorical_columns)}")
print(f"  • One-Hot Encoding aplicado a: {len(low_cardinality)} variables")
print(f"  • Label Encoding aplicado a: {len(high_cardinality)} variables")

if variables_to_normalize:
    print(f"\n📏 NORMALIZACIÓN APLICADA:")
    for col in variables_to_normalize:
        method = normalization_strategy[col]['method']
        print(f"  • {col}: {method}")

print(f"\n📁 DIVISIÓN DE DATOS:")
print(f"  • Conjunto de entrenamiento: {len(X_train):,} registros ({len(X_train)/len(X)*100:.1f}%)")
print(f"  • Conjunto de prueba: {len(X_test):,} registros ({len(X_test)/len(X)*100:.1f}%)")
print(f"  • Variable objetivo: {target_column}")
print(f"  • Semilla utilizada: {random_seed}")

print(f"\n💾 ARCHIVOS GENERADOS:")
print(f"  • {train_features_path}")
print(f"  • {train_target_path}")
print(f"  • {test_features_path}")
print(f"  • {test_target_path}")
if encoders:
    print(f"  • {encoders_path}")
if scalers:
    print(f"  • {scalers_path}")
print(f"  • {metadata_path}")

print(f"\n✅ Los datos están listos para el entrenamiento de modelos de machine learning!")

🎉 PREPARACIÓN DE DATOS COMPLETADA EXITOSAMENTE
📊 ESTADÍSTICAS GENERALES:
  • Conjunto de datos original: (113999, 16)
  • Conjunto de datos normalizado: (113999, 16)
  • Conjunto de datos codificado: (113999, 19)
  • Nuevas características creadas: 3

🔧 PROCESAMIENTO REALIZADO:
  • Variables numéricas: 11
  • Variables normalizadas: 3
  • Variables categóricas procesadas: 4
  • One-Hot Encoding aplicado a: 2 variables
  • Label Encoding aplicado a: 2 variables

📏 NORMALIZACIÓN APLICADA:
  • duration_ms: StandardScaler
  • loudness: RobustScaler
  • tempo: StandardScaler

📁 DIVISIÓN DE DATOS:
  • Conjunto de entrenamiento: 91,199 registros (80.0%)
  • Conjunto de prueba: 22,800 registros (20.0%)
  • Variable objetivo: popularity
  • Semilla utilizada: 20250901

💾 ARCHIVOS GENERADOS:
  • ../data/train/X_train.csv
  • ../data/train/y_train.csv
  • ../data/test/X_test.csv
  • ../data/test/y_test.csv
  • ../data/train/encoders.joblib
  • ../data/train/scalers.joblib
  • ../data/train/metada